In [26]:
!pip3 install torch
!pip3 install pandas

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [27]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [28]:
f = r"data/fullset_test.csv"
df = pd.read_csv(f, names=["name", "seq", "class"])

In [29]:
mapping = {'A': 0, 'T': 1, 'C': 2, 'G': 3}

In [30]:
def mapping_fn(string):
    x = [mapping[x] for x in string]
    return x

In [31]:
data = df["seq"].apply(lambda x: mapping_fn(x))
data2 = np.zeros((len(df), 300), dtype=np.int64)

In [32]:
for i, d in enumerate(data):
    data2[i, :] = d

In [33]:
data2.shape

(26405, 300)

In [34]:
torch_tensor_output = torch.tensor(df['class'].values, dtype=torch.float)
torch_tensor_vectors = torch.from_numpy(data2)

In [35]:
torch_tensor_vectors.dtype

torch.int64

In [36]:
torch_tensor_vectors.shape

torch.Size([26405, 300])

In [37]:
EMBEDDING_VEC_SIZE = 32
DNA_BASES = 4

In [38]:
class Example(nn.Module):
    def __init__(self):
        super(Example, self).__init__()
        self.embed = nn.Embedding(DNA_BASES, EMBEDDING_VEC_SIZE)
        self.fc = nn.Linear(300 * 32, 1)
    
    def forward(self, x):
        x = self.embed(x)
        x = x.flatten()
        x = self.fc(x)
        return torch.sigmoid(x)

In [39]:
net = Example()

In [40]:
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [41]:
for epoch in range(5):  # loop over the dataset multiple times
    epoch_loss = 0.0
    for i, (inputs, labels) in enumerate(zip(torch_tensor_vectors, torch_tensor_output)):

        # zero the parameter gradients
        optimizer.zero_grad()

        labels = labels.view(1)
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        epoch_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, epoch_loss / (i + 1)))

print('Finished Training')

[1,  2000] loss: 1.007
[1,  4000] loss: 0.852
[1,  6000] loss: 0.931
[1,  8000] loss: 0.950
[1, 10000] loss: 0.966
[1, 12000] loss: 1.018
[1, 14000] loss: 1.134
[1, 16000] loss: 1.171
[1, 18000] loss: 1.164
[1, 20000] loss: 1.166
[1, 22000] loss: 1.160
[1, 24000] loss: 1.180
[1, 26000] loss: 1.191
[2,  2000] loss: 1.041
[2,  4000] loss: 0.917
[2,  6000] loss: 0.983
[2,  8000] loss: 0.981
[2, 10000] loss: 1.086
[2, 12000] loss: 1.103
[2, 14000] loss: 1.171
[2, 16000] loss: 1.171
[2, 18000] loss: 1.155
[2, 20000] loss: 1.151
[2, 22000] loss: 1.162
[2, 24000] loss: 1.186
[2, 26000] loss: 1.210
[3,  2000] loss: 1.195
[3,  4000] loss: 1.148
[3,  6000] loss: 1.203
[3,  8000] loss: 1.301
[3, 10000] loss: 1.332
[3, 12000] loss: 1.352
[3, 14000] loss: 1.421
[3, 16000] loss: 1.449
[3, 18000] loss: 1.419
[3, 20000] loss: 1.408
[3, 22000] loss: 1.416
[3, 24000] loss: 1.449
[3, 26000] loss: 1.477
[4,  2000] loss: 1.265
[4,  4000] loss: 1.247
[4,  6000] loss: 1.358
[4,  8000] loss: 1.409
[4, 10000] 

In [42]:
class seqData(Dataset):   
    def __init__(self, sequences, labels):
        self.data = torch.from_numpy(sequences)
        self.labels = torch.tensor(labels, dtype=torch.float)
        
    def __len__(self):
        return len(self.data)   
    
    def __getitem__(self, index):
        label = self.labels[index]
        data_val = self.data[index]
        return data_val,label

In [43]:
train_dataset = seqData(data2, df['class'].values)

In [44]:
train_dataset.__getitem__(2)

(tensor([1, 3, 0, 3, 2, 0, 2, 0, 1, 0, 1, 2, 1, 3, 1, 3, 3, 3, 3, 2, 0, 3, 3, 1,
         3, 3, 3, 3, 0, 0, 1, 0, 0, 0, 2, 0, 3, 2, 0, 2, 0, 0, 1, 1, 0, 2, 0, 1,
         0, 3, 2, 1, 0, 3, 1, 0, 1, 1, 1, 1, 3, 2, 2, 1, 1, 1, 1, 1, 2, 0, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 3, 1, 1, 1, 0, 2, 2, 1, 2, 0, 1, 0, 1,
         1, 1, 1, 0, 1, 0, 0, 0, 2, 1, 1, 0, 1, 3, 1, 0, 0, 3, 1, 1, 3, 3, 2, 1,
         1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 3, 2, 0, 0, 1, 0, 1, 0, 0, 0,
         0, 2, 0, 0, 0, 0, 0, 0, 1, 3, 2, 1, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 3,
         2, 0, 0, 1, 3, 3, 1, 1, 1, 1, 2, 0, 0, 3, 0, 1, 3, 1, 3, 3, 1, 2, 1, 2,
         0, 0, 0, 3, 1, 3, 1, 3, 0, 1, 1, 1, 2, 2, 0, 3, 3, 3, 3, 1, 1, 2, 2, 2,
         0, 0, 3, 0, 2, 2, 2, 1, 1, 2, 0, 3, 0, 1, 2, 1, 3, 2, 0, 0, 3, 3, 1, 2,
         0, 0, 1, 1, 1, 3, 1, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 1, 3, 0, 1, 3, 1, 1,
         0, 2, 2, 1, 2, 3, 1, 3, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 0, 2, 1, 2, 1, 1,
         1, 2, 0, 2, 0, 0, 3

In [45]:
BAT_SIZE = 1
train_loader = DataLoader(train_dataset, batch_size=BAT_SIZE, shuffle=True)

In [46]:
for epoch in range(5):  # loop over the dataset multiple times
    epoch_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):

        # zero the parameter gradients
        optimizer.zero_grad()

        #labels = labels.view(1)
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        epoch_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, epoch_loss / (i + 1)))

print('Finished Training')

[1,  2000] loss: 1.865
[1,  4000] loss: 1.774
[1,  6000] loss: 1.619
[1,  8000] loss: 1.604
[1, 10000] loss: 1.604
[1, 12000] loss: 1.654
[1, 14000] loss: 1.617
[1, 16000] loss: 1.598
[1, 18000] loss: 1.549
[1, 20000] loss: 1.540
[1, 22000] loss: 1.531
[1, 24000] loss: 1.518
[1, 26000] loss: 1.533
[2,  2000] loss: 1.627
[2,  4000] loss: 1.627
[2,  6000] loss: 1.662
[2,  8000] loss: 1.638
[2, 10000] loss: 1.756
[2, 12000] loss: 1.732
[2, 14000] loss: 1.707
[2, 16000] loss: 1.699
[2, 18000] loss: 1.700
[2, 20000] loss: 1.684
[2, 22000] loss: 1.668
[2, 24000] loss: 1.613
[2, 26000] loss: 1.583
[3,  2000] loss: 1.335
[3,  4000] loss: 1.507
[3,  6000] loss: 1.532
[3,  8000] loss: 1.585
[3, 10000] loss: 1.615
[3, 12000] loss: 1.543
[3, 14000] loss: 1.537
[3, 16000] loss: 1.606
[3, 18000] loss: 1.619
[3, 20000] loss: 1.601
[3, 22000] loss: 1.629
[3, 24000] loss: 1.619
[3, 26000] loss: 1.609
[4,  2000] loss: 1.445
[4,  4000] loss: 1.448
[4,  6000] loss: 1.476
[4,  8000] loss: 1.595
[4, 10000] 